# Exercise_1

Basic idea: Choose scenario with click on a button in the GUI. Then click on a Start button to run the simulation. \
    In each timestep of the simulation the new (precise) position of each pedestrian is calculated. Afterwards the discretized position in the grid/cellular automaton is shown in the GUI.

In [1]:
import tkinter as tk
import numpy as np # not used currently
import math
import time # not used currently, but can be helpful for drawing after each timestep

In [2]:
class Person():
    '''
    pos_x, pos_y: precise position of pedestrian (Idea: cells are discretized positions we only use for drawing)
    speed: movement speed of pedestrian in m/s
    move(): see below
    '''
    def __init__(self, position_x, position_y, speed=1):
        self.pos_x = position_x
        self.pos_y = position_y
        self.speed = speed
        
    def move(self, grid, scenario):
        '''
        equals an update for the position depending on speed, interaction of pedestrians and obstacle avoidance
        costField is discritized per se, so position needs to be discretized as well
        looks for minimal cost in all directions within the feasible area
        '''
        costField = scenario.costField + pedestrianInteraction(scenario) #+ obstacleAvoidance
        (x_opt, y_opt) = (self.pos_x, self.pos_y)
        cost_min = scenario.costField[int(self.pos_x)][int(self.pos_y)]
        if pos_x-speed >= 0:#checks whether end point of motion is within existing area
            #discretize x and y, then check cost and if desired cell is empty
            dx = int(self.pos_x-self.speed)
            dy = int(self.pos_y)
            if scenario.costField[dx][dy] < cost_min and grid.grid[dx][dy]=='E':
                (x_opt, y_opt) = (self.pos_x-self.speed, self.pos_y)#not discretized
                cost_min = scenario.costField[dx][dy]
        if pos_y+speed < scenario.columns:
            dx = int(self.pos_x)
            dy = int(self.pos_y+self.speed)
            if scenario.costField[dx][dy] < cost_min and grid.grid[dx][dy]=='E':
                (x_opt, y_opt) = (self.pos_x, self.pos_y+self.speed)
                cost_min = scenario.costField[dx][dy]
        if pos_x+speed < scenario.rows:
            dx = int(self.pos_x+self.speed)
            dy = int(self.pos_y)
            if scenario.costField[dx][dy] < cost_min and grid.grid[dx][dy]=='E':
                (x_opt, y_opt) = (self.pos_x+self.speed, self.pos_y)
                cost_min = scenario.costField[dx][dy]
        if pos_y-speed >= 0:
            dx = int(self.pos_x)
            dy = int(self.pos_y-self.speed)
            if scenario.costField[dx][dy] < cost_min and grid.grid[dx][dy]=='E':
                (x_opt, y_opt) = (self.pos_x, self.pos_y-self.speed)
                cost_min = scenario.costField[dx][dy]
        #TODO: add diagonal movement with sqrt(2) somewhere and maybe sin/cos stuff
        
        #updates grid for drawing and own position
        grid.grid[int(self.pos_x)][int(self.pos_y)]='E'
        grid.grid[int(x_opt)][int(y_opt)]='P'
        self.pos_x = x_opt
        self.pos_y = y_opt
        
        
    def pedestrianInteraction(self, scenario):
        self.interactionField = [[0.0 for y in range(scenario.columns)] for x in range(scenario.rows)]
        for p in scenario.persons:
            if p == self:
                continue#this is to avoid penalty with yourself. I assume you like yourself :)
            else:
                # TODO: rmax -> add with fixed value in scenario
                x = int(p.pos_x)
                y = int(p.pos_y)
                self.interactionField[x][y] = math.exp(1/(self.rmax**2))
                for r in range(self.rmax):
                    #TODO: diagonal be careful
                    if x-r >= 0:
                        self.interactionField[x-r][y] += math.exp(-1/(r**2 - scenario.rmax**2))
                    if y+r < scenario.columns:
                        self.interactionField[x][y+r] += math.exp(-1/(r**2 - scenario.rmax**2))
                    if x+r < scenario.rows:
                        self.interactionField[x+r][y] += math.exp(-1/(r**2 - scenario.rmax**2))
                    if y-r >= 0:
                        self.interactionField[x][y-r] += math.exp(-1/(r**2 - scenario.rmax**2))

In [3]:
class Scenario():
    '''
    creates persons, targets, obstacles and grid size depending on the scenario
    '''
    def __init__(self):
        self.rows = 25
        self.columns = 25
        self.persons = []
        self.targets = []
        self.obstacles = []
        self.grid = Grid(self)
        #self.setTo1()
        
    def setTo1(self):#one person straight
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo2(self):#five persons in a circle
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25), Person(25, 35), Person(25, 15), Person(40, 30), Person(40, 20)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo3(self):#bottleneck
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25), Person(25, 35), Person(25, 15), Person(40, 30), Person(40, 20)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo4(self):#chickentest
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25), Person(25, 35), Person(25, 15), Person(40, 30), Person(40, 20)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    #TODO: addd RiMEA tests
    #TODO: change persons, obstacles and maybe targets for scenarios 3 and 4
        
    def setupCostField(self):
        #cost scaled by max possible cost/distance
        self.costField = [[1.0 for y in range(self.columns)] for x in range(self.rows)]
        max_dist = math.sqrt(self.rows**2 + self.columns**2)
        for x in range(self.rows):
            for y in range(self.columns):
                if (x,y) in self.targets:
                    self.costField[x][y] = 0
                else:#computes cost/distance for nearest target
                    dist = max_dist
                    for t in self.targets:
                        min_dist = math.sqrt((x - t[0])**2 + (y - t[1])**2)
                        if min_dist < dist:
                            dist = min_dist
                    cost = dist/max_dist
                    self.costField[x][y] = cost
                    
    def move(self):
        for p in self.persons:
            p.move(grid, self)

In [4]:
class Grid():
    '''
    Basically only for visualization.
    '''
    def __init__(self, scenario):
        self.scenario = scenario
        self.grid = [['E' for y in range(scenario.columns)] for x in range(scenario.rows)]
        for p in scenario.persons:
            self.grid[p.pos_x][p.pos_y] = 'P'
        for t in scenario.targets:
            self.grid[t[0]][t[1]] = 'T'
        for o in scenario.obstacles:
            self.grid[o[0]][o[1]] = 'O'
        
    def draw(self, master):
        '''
        Sould draw the grid and the persons/targets/obstacles.
        Depends on tkinter stuff. I have to look at this in more detail.
        '''
        

In [9]:
def main():
    '''
    Setup of the interface stuff.
    When a scenario is selected via a button, it should appear next to the buttons in the canvas.
    When clicking on start button, the current scenario should be siumlated, i.e. for every timestep=1s each pedestrian should move() and the drawing should be renewed.
    
    '''
    window = tk.Tk()
    window.title('Exercise 1')
    
    #I'm not sure, whether a canvas is the best option, perhaps a frame is better.
    canvas = tk.Canvas(master=window)
    canvas.pack(side='left')
    
    scenario = Scenario()
    
    btnScenario1 = tk.Button(master=window, text='Scenario 1', bg='#26f9ad', width='8', command=None)
    btnScenario1.pack()#(side=RIGHT)
    btnScenario2 = tk.Button(master=window, text='Scenario 2', bg='#26f9ad', width='8', command=None)
    btnScenario2.pack()#(side=RIGHT)
    btnScenario3 = tk.Button(master=window, text='Scenario 3', bg='#26f9ad', width='8', command=None)
    btnScenario3.pack()#(side=RIGHT)
    btnScenario4 = tk.Button(master=window, text='Scenario 4', bg='#26f9ad', width='8', command=None)
    btnScenario4.pack()#(side=RIGHT)
    
    #TODO: add buttons for RiMEA tests
    
    #Below is an attempt to display explanations for the scenrio buttons. As this is rather cosmetic I suggest to do it at the end.
    #explanation = tk.Text(master=window, height=4, state=DISABLED)
    #explanation.insert(INSERT, "First scenario is one pedestrain walking on a straight line to the target.\n Second scenario is five pedestrians in a circle around the target.\n Third scenario is bottleneck.\n Fourth scenario is chicken test.\n")
    #explanation.pack(side=BOTTOM)
    
    #TODO: call start() (not yet implemented) or another function as starting point
    #TODO: setup timed loop to call scenario.move() (until max timesteps reached)
    #TODO: call draw() in Grid. How? Maybe add grid to scenario
    
    window.mainloop()

In [10]:
if __name__ == "__main__":
    main()